In [18]:
import cv2

cap = cv2.VideoCapture("people.mp4")

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sub = cv2.createBackgroundSubtractorMOG2()

video = cv2.VideoWriter('traffic_counter.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (width, height), 1)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    fgmask = sub.apply(frame)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closing = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    dilation = cv2.dilate(opening, kernel)

    retvalbin, bins = cv2.threshold(dilation, 220, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(bins, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    min_area = 400
    max_area = 50000
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area < area < max_area:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    video.write(frame)

    cv2.imshow("Contours", frame)
    
    key = cv2.waitKey(100)
    if key == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()
video.release()
